In [1]:
import mujoco
import numpy as np
import pinocchio as pin

from robot_descriptions.z1_description import URDF_PATH
from robot_descriptions.z1_mj_description import MJCF_PATH

# from robot_descriptions.skydio_x2_description import URDF_PATH
# from robot_descriptions.skydio_x2_mj_description import MJCF_PATH
from mujoco_sysid import regressors, parameters

In [2]:
mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
# enable energy
mjmodel.opt.enableflags |= mujoco.mjtEnableBit.mjENBL_ENERGY
# disable friction
mjmodel.opt.disableflags |= (
    mujoco.mjtDisableBit.mjDSBL_CONTACT | mujoco.mjtDisableBit.mjDSBL_FRICTIONLOSS | mujoco.mjtDisableBit.mjDSBL_LIMIT
)
mjdata = mujoco.MjData(mjmodel)

pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pin.Data(pinmodel)

In [3]:
# np.random.seed(10)

In [4]:
q, v = np.random.randn(mjmodel.nq), np.ones(mjmodel.nv)

In [5]:
mjdata.qpos[:] = q.copy()
mjdata.qvel[:] = v.copy()

mujoco.mj_step(mjmodel, mjdata)

np.sum(mjdata.energy)

1.0995959454222115

In [6]:
np.sum([pin.computePotentialEnergy(pinmodel, pindata, q), pin.computeKineticEnergy(pinmodel, pindata, q, v)])

0.9839147234384552

In [7]:
mjmodel.jnt_bodyid, list(range(2, 8))

(array([2, 3, 4, 5, 6, 7], dtype=int32), [2, 3, 4, 5, 6, 7])

In [8]:
theta = np.concatenate([parameters.get_dynamic_parameters(mjmodel, i) for i in mjmodel.jnt_bodyid])

theta.shape, theta[:10]

((60,),
 array([ 6.73326000e-01,  1.66311522e-06, -1.69664685e-04,  1.56021081e-02,
         1.64484975e-03, -5.95801269e-08,  1.08083746e-03, -4.38536417e-07,
         4.43147004e-06,  8.39403034e-04]))

In [9]:
reg_en = regressors.mj_energyRegressor(mjmodel, mjdata)[2] @ theta

reg_en

spatial [0.         0.         0.         0.         0.         1.25028273] [0. 0. 1. 0. 0. 0.]
spatial [ 0.          0.          0.          1.04313506  1.29695411 -0.68925769] [ 0.83431934  1.         -0.55128146  0.          0.          0.        ]
spatial [-0.15277738  0.24124019  0.42745186  1.21425819  2.13724787 -0.29796635] [ 0.97118689  2.         -0.23831917 -0.11779706  0.19294851  0.32958133]
spatial [-0.01045596  0.10707081 -0.04825598  1.21916593  2.861868    0.2772027 ] [ 0.97511219  3.          0.22171201  0.04424964  0.08563728 -0.09685722]
spatial [-0.11956464  0.04253879 -0.24858674 -2.10618843  2.289241    1.57652568] [-2.33254461  2.12369476  1.22171201 -0.07397042  0.08197055 -0.30685722]
spatial [-0.11956464  0.29600692 -0.23933835 -0.80661412  1.07692499  2.56247734] [-1.33254461  1.12896659  2.17441811 -0.07397042  0.34155282 -0.26968707]


0.9839142668298118

In [10]:
params = []

for i in range(len(pinmodel.inertias) - 1):
    params.extend(pinmodel.inertias[i + 1].toDynamicParameters())

params = np.array(params)

params.shape, params[:10]

((60,),
 array([ 6.73325510e-01,  1.66311401e-06, -1.69664562e-04,  1.56020900e-02,
         1.64484954e-03, -5.95809285e-08,  1.08083680e-03, -4.38537162e-07,
         4.43141463e-06,  8.39402756e-04]))

In [11]:
dummy_idx = 2
p1 = params[dummy_idx * 10 : (dummy_idx + 1) * 10]
p2 = theta[dummy_idx * 10 : (dummy_idx + 1) * 10]

p1, p2, np.linalg.norm(p1 - p2)

(array([ 8.39408740e-01,  8.90546192e-02, -4.54804246e-03,  2.91810627e-02,
         2.11969748e-03,  3.95821285e-04,  2.00048153e-02, -5.17689964e-03,
         1.44787375e-04,  1.83388418e-02]),
 array([ 8.39409000e-01,  8.90545796e-02, -4.54804387e-03,  2.91810466e-02,
         2.11969530e-03,  3.95805704e-04,  2.00047984e-02, -5.17689303e-03,
         1.44783395e-04,  1.83388273e-02]),
 2.650078190181762e-07)

In [12]:
def kinetic_regressor(q, v):
    pin.computeAllTerms(pinmodel, pindata, q, v)
    regressor = np.zeros((1, (pinmodel.nbodies - 1) * 10))
    for i in range(pinmodel.nbodies - 1):
        vel = pin.getVelocity(pinmodel, pindata, i + 1, pin.LOCAL)
        vl = vel.linear
        va = vel.angular
        print(vl, va)

        regressor[0, i * 10 + 0] = 0.5 * (vl[0] ** 2 + vl[1] ** 2 + vl[2] ** 2)
        regressor[0, i * 10 + 1] = -va[1] * vl[2] + va[2] * vl[1]
        regressor[0, i * 10 + 2] = va[0] * vl[2] - va[2] * vl[0]
        regressor[0, i * 10 + 3] = -va[0] * vl[1] + va[1] * vl[0]
        regressor[0, i * 10 + 4] = 0.5 * va[0] ** 2
        regressor[0, i * 10 + 5] = va[0] * va[1]
        regressor[0, i * 10 + 6] = 0.5 * va[1] ** 2
        regressor[0, i * 10 + 7] = va[0] * va[2]
        regressor[0, i * 10 + 8] = va[1] * va[2]
        regressor[0, i * 10 + 9] = 0.5 * va[2] ** 2

    return regressor

In [13]:
(
    kinetic_regressor(q, v) @ theta,
    kinetic_regressor(q, v) @ params,
)

[0. 0. 0.] [0. 0. 1.]
[0. 0. 0.] [ 0.83431934  1.         -0.55128146]
[-0.11779706  0.19294851  0.32958133] [ 0.97118689  2.         -0.23831917]
[ 0.04424964  0.08563728 -0.09685722] [0.97511219 3.         0.22171201]
[-0.07397042  0.08197055 -0.30685722] [-2.33254461  2.12369476  1.22171201]
[-0.07397042  0.34155282 -0.26968707] [-1.33254461  1.12896659  2.17441811]
[0. 0. 0.] [0. 0. 1.]
[0. 0. 0.] [ 0.83431934  1.         -0.55128146]
[-0.11779706  0.19294851  0.32958133] [ 0.97118689  2.         -0.23831917]
[ 0.04424964  0.08563728 -0.09685722] [0.97511219 3.         0.22171201]
[-0.07397042  0.08197055 -0.30685722] [-2.33254461  2.12369476  1.22171201]
[-0.07397042  0.34155282 -0.26968707] [-1.33254461  1.12896659  2.17441811]


(array([0.15232524]), array([0.15232525]))